In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.callbacks import EarlyStopping
from keras.layers import Dense , LSTM , Dropout , Input , Embedding , Activation
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from matplotlib import pyplot as plt
from random import randint

In [12]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/spam.csv', delimiter = ',' , encoding = 'latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
df = pd.read_csv('spam.csv' , delimiter = ',' , encoding = 'latin-1')

FileNotFoundError: ignored

In [13]:
df.drop(['Unnamed: 2' ,'Unnamed: 3', 'Unnamed: 4'] , axis  = 1 ,inplace = True  )
df.info 

<bound method DataFrame.info of         v1                                                 v2
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham              Will Ì_ b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name

[5572 rows x 2 columns]>

In [14]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [15]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
print(Y)
print(Y.shape)
Y = Y.reshape(-1,1)
print(Y.shape)

[0 0 1 ... 0 0 0]
(5572,)
(5572, 1)


In [16]:
X_train , X_test , Y_train , Y_test = train_test_split(X,Y,test_size = 0.15)

In [17]:
max_words = 1000
max_len =150
tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences , maxlen = max_len )

In [28]:
def RNN():
  inputs = Input(name = 'inputs',shape = [max_len])
  layer =  Embedding(max_words , 50 , input_length = max_len)(inputs)
  layer = LSTM(64)(layer)
  layer = Dense(256, name = 'FC1')(layer)
  layer = Activation('relu')(layer)
  layer =Dropout(0.5)(layer)
  layer = Dense(1, name = 'out_layer')(layer)
  layer = Activation('sigmoid')(layer)
  model = Model(inputs = inputs , outputs = layer)
  return model

In [27]:
model = RNN()
model.summary()
model.compile(loss = 'binary_crossentropy' , optimizer = RMSprop() , metrics = ['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257   

In [31]:
model.fit(sequences_matrix , Y_train , batch_size = 128 ,  epochs = 10 , validation_split = 0.2 , callbacks = [EarlyStopping(monitor = 'val_loss')])

Epoch 1/10
30/30 [==============================] - 8s 182ms/step - loss: 0.4392 - accuracy: 0.8213 - val_loss: 0.1439 - val_accuracy: 0.9873
Epoch 2/10
30/30 [==============================] - 5s 164ms/step - loss: 0.0993 - accuracy: 0.9779 - val_loss: 0.0471 - val_accuracy: 0.9905
Epoch 3/10
30/30 [==============================] - 5s 162ms/step - loss: 0.0453 - accuracy: 0.9889 - val_loss: 0.0379 - val_accuracy: 0.9926
Epoch 4/10
30/30 [==============================] - 5s 165ms/step - loss: 0.0238 - accuracy: 0.9942 - val_loss: 0.0406 - val_accuracy: 0.9916


In [32]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences , maxlen = max_len)

In [33]:
accr = model.evaluate(test_sequences_matrix , Y_test)

27/27 [==============================] - 0s 18ms/step - loss: 0.0457 - accuracy: 0.9868


In [34]:
print('Test set\n Loss: {:0.3f}\n Accuracy: {:0.3f}'.format(accr[0] , accr[1]))

Test set
 Loss: 0.046
 Accuracy: 0.987
